# Machine Learning Crash Course: H2O AutoML

This is a [Jupyter](https://jupyter.org/) Notebook. When you execute code within the notebook, the results appear beneath the code. To execute a code chunk, place your cursor on the cell and press *Shift+Enter*.

### Start H2O

Import the **h2o** Python module and `H2OAutoML` class and initialize a local H2O cluster.

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

### Load Data

We're going to use a subset of a publically available [Product Backorders](https://www.kaggle.com/tiredgeek/predict-bo-trial/data) dataset.  The goal here is to predict whether or not a product will be put on backorder status, given a number of product metrics such as current inventory, transit time, demand forecasts and prior sales.

In [ ]:
# Use local data file or download from GitHub
data_path = "https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/product_backorders.csv"

# Load data into H2O
df = h2o.import_file(data_path)

We're going to be predicting one of the specific columns of our dataset. Specifically, one that indicates whether a given product was put on backorder or not. Let's inspect the dataset we've loaded, which was returned to us as a "data frame".

In [ ]:
df

We can also investigate the distributions and types of the columns we have.

In [ ]:
df.describe()

Next, let's identify the response & predictor columns by saving them as `x` and `y`.  The `"sku"` column is a unique identifier so we'll want to remove that from the set of our predictors.

In [ ]:
y = "went_on_backorder"
x = df.columns
x.remove(y)
x.remove("sku")

## Build Our Model

We're going to run H2O's AutoML, and ask it to build at most 10 models to try and predict our outcome.

In [ ]:
automl = H2OAutoML(max_models=5)
automl.train(x=x, y=y, training_frame=df)

## Leaderboard

AutoML has built us multiple slightly different models in an attempt to predict the outcome we asked it to learn. Which model is the best, and how good is it?

H2O ranks the models it has built for us in an *AutoML Leaderboard*, a ranked list of the models it has built, sorted by a *metric* that indicates how good each model is.

The leader model is stored at `automl.leader` and the leaderboard is stored at `automl.leaderboard`.

In [ ]:
leaderboard = automl.leaderboard
leader = automl.leader

Now we will view a snapshot of the top models.  Here we should see the two Stacked Ensembles at or near the top of the leaderboard.  Stacked Ensembles can almost always outperform a single model.

In [ ]:
leaderboard.head()

To view the entire leaderboard, specify the `rows` argument of the `head()` method as the total number of rows:

In [ ]:
leaderboard.head(rows=leaderboard.nrows)

## Making Predictions

Now that we have a model, we can ask the model for predictions on a new data point.

In [ ]:
automl.leader.predict(
    h2o.H2OFrame(
        [
            [12345678, 13, 30, 20, 80, 100, 130, 50, 20, 100, 32, 0, False, 0, 100, 80, 100, False, False, False, False, False],
[12345678234, 1223, 30, 20, 80, 1030, 130, 50, 20, 100, 321, 0, False, 0, 100, 80, 100, False, False, False, False, False],
        ],
        column_names=["sku", "national_inv", "lead_time", "in_transit_qty", "forecast_3_month", "forecast_6_month", "forecast_9_month", "sales_1_month", "sales_3_month", "sales_6_month", "sales_9_month", "min_bank", "potential_issue", "pieces_past_due", "perf_6_month_avg", "perf_12_month_avg", "local_bo_qty", "deck_risk", "oe_constraint", "ppap_risk", "stop_auto_buy", "rev_stop"],
    ),
)

## Save Leader Model

Once we have trained a model, and decided we want to keep it (or keep the leader model), we can save that model for reuse later.

In [ ]:
h2o.save_model(automl.leader, path = "./product_backorders_model_bin")

The contents of this notebook were lightly adapted from a tutorial in the [official H2O AutoML documentation](http://docs.h2o.ai/h2o-tutorials/latest-stable/h2o-world-2017/automl/index.html). There are many more interesting tutorials to work through there. Explore them!